# Slide 15 — Class visualization (maximize a logit)

Gradient‑ascent on the **input** to maximize a chosen **class logit** (ImageNet).


In [4]:
import sys
ROOT = "/Users/juliawenkmann/Documents/CodingProjects/damien/sophia_summit/sophia_summit_presentation/src/lucent"
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from lucent.modelzoo import inceptionv1
from lucent.optvis import render, objectives
import torchvision

### Helper to find class logit by name

In [5]:
def imagenet_index(query: str) -> int:
    import difflib
    categories = None
    # Try a few torchvision weights that include the 'categories' list
    try:
        from torchvision.models import ResNet50_Weights
        categories = ResNet50_Weights.IMAGENET1K_V2.meta["categories"]
    except Exception:
        try:
            from torchvision.models import GoogLeNet_Weights
            categories = GoogLeNet_Weights.IMAGENET1K_V1.meta["categories"]
        except Exception:
            pass
    if categories is None:
        raise RuntimeError("Could not load ImageNet category names from torchvision. Please pass a numeric class ID.")
    hits = [i for i, name in enumerate(categories) if query.lower() in name.lower()]
    if hits:
        return hits[0]
    # fallback to closest name
    best = difflib.get_close_matches(query, categories, n=1)
    if not best:
        raise ValueError(f"No class matching '{query}'.")
    return categories.index(best[0])

# Example:
class_id = imagenet_index("golden retriever")
obj = objectives.channel("labels", class_id)
print(class_id)

207


### Create Snapshots while optimizing for class logit - without bias

In [ ]:

# 1) Model
model = inceptionv1.inceptionv1(pretrained=True).eval()

# 2) Pick a class (integer ID in [0..999], e.g., 207 ≈ "golden retriever" in ImageNet)
class_id = 207  # <-- change me or use the helper above to search by name

# 3) Objective: maximize that class logit
obj = objectives.channel("labels", class_id)

# 4) Render with snapshots and export PDF
pdf_out = "/Users/juliawenkmann/Documents/CodingProjects/damien/sophia_summit/sophia_summit_presentation/figures/class_207_progress.pdf"

imgs = render.render_feature_with_snapshots(
    model,
    obj,
    snapshot_steps=[0, 4, 48, 2048],
    img_size=224,
    device="auto",   
    pdf_path=pdf_out,
    left_text=f"Optimize an input to increase the logit for class {class_id}.",
)
print("Saved:", pdf_out)

100%|██████████| 2048/2048 [1:11:40<00:00,  2.10s/it]


Saved: /Users/juliawenkmann/Documents/CodingProjects/damien/sophia_summit/sophia_summit_presentation/figures/class_207_progress.pdf


### Create Snapshots while optimizing for class logit - with bias

In [ ]:
pdf_out = "/Users/juliawenkmann/Documents/CodingProjects/damien/sophia_summit/sophia_summit_presentation/figures/class_207_progress_random_noise.pdf"

imgs = render.render_feature_with_snapshots(
    model,
    obj,
    snapshot_steps=[0, 4, 48, 2048],
    img_size=224,
    device="auto",   
    pdf_path=pdf_out,
    left_text=f"Optimize an input to increase the logit for class {class_id}.",
)
print("Saved:", pdf_out)